In [203]:
import pandas as pd
import numpy as np

In [190]:
data = pd.read_csv('train.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [191]:
del data['Name'], data['Sex'], data['Age'], data['Ticket'], data['Cabin'], data['Embarked']

In [194]:
# Построим модель, которая будет предсказывать выживет пассажир или нет
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
model = LogisticRegression(max_iter=1000)
y = data['Survived']
data.drop('Survived', axis=1)
x = data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [195]:
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

1.0

In [196]:
data_new = pd.read_csv('train.csv')
# У каждого пассажира свой уникальный идентификатор, поэтому это никак не может повлиять на модель
# Так как имя никак не влияет на то выживет пассажир или нет, этот столб тоже можно удалить
# Номер билета не нужен, так как он коррелирует с классом пассажира
# Стоимость билета, так как она коррелирует с классом пассажира
# Порт посадки, так как крушение произошло после того, как все пассажиры поднялись на борт
del data_new['PassengerId'], data_new['Name'], data_new['Ticket'], data_new['Fare'], data_new['Embarked']

In [197]:
import math
count = 0
for i in range(891):
  if math.isnan(data_new['Age'][i]) or data_new['Cabin'][i] is np.NaN:
    count += 1
count / 891
# Как видно, если бы мы удалили все объекты с пропусками, мы бы потеряли 79.2 % всех данных

0.792368125701459

In [198]:
mean_age = data_new['Age'].mean()
data_new['Age'] = data_new['Age'].fillna(mean_age)
data_new['Cabin'] = data_new['Cabin'].fillna('NoCabin')

In [199]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_new['Sex'] = le.fit_transform(data_new['Sex'])
data_new['Cabin'] = le.fit_transform(data_new['Cabin'])


In [200]:
data_new.max(), data_new.min(), data_new.info()
# Выбросов не было выявлено, так как все значения находятся в пределах нормы, а также типы используемых данных тоже являются нормой

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Cabin     891 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 48.9 KB


(Survived      1.0
 Pclass        3.0
 Sex           1.0
 Age          80.0
 SibSp         8.0
 Parch         6.0
 Cabin       147.0
 dtype: float64, Survived    0.00
 Pclass      1.00
 Sex         0.00
 Age         0.42
 SibSp       0.00
 Parch       0.00
 Cabin       0.00
 dtype: float64, None)

In [214]:
mas = []
for i in range(891):
  if data_new['Age'][i] <= 12:
    mas.append('Child')
  elif data_new['Age'][i] <= 18:
    mas.append('Teen')
  elif data_new['Age'][i] <= 60:
    mas.append('Adult')
  else:
    mas.append('Elderly')
data_new['Age_Category'] = mas
del data_new['Age']
data_new['Age_Category'] = le.fit_transform(data_new['Age_Category'])

In [215]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Survived      891 non-null    int64
 1   Pclass        891 non-null    int64
 2   Sex           891 non-null    int64
 3   SibSp         891 non-null    int64
 4   Parch         891 non-null    int64
 5   Cabin         891 non-null    int64
 6   Age_Category  891 non-null    int64
dtypes: int64(7)
memory usage: 48.9 KB


In [216]:
y = data_new['Survived']
data_new.drop('Survived', axis=1)
x = data_new
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

1.0

Был загружен тестовый датасет
Сначала были удалены все столбцы содержащие нечисловые знаечния или nan

После была обучена модель, определяющая выживет пассажир или нет.
Метрика roc_auc_score показала 1, что значит, что модель идеально точно предсказала все значения целевого столбца.

Далее данные были загружены заново, после чего были исключены столбца, которые, по-моему мнению, никак не влияют на целевой столбец.

Был посчитан общий процент потенциально-потрянных данных при удалении всех объектов, содержащих значение nan.

Для борьбы с этим значения nan в столбце Age были заменены на среднее значение, а в столбце Cabin - на 'NoCabin', так как на титанике не все имели кабину.

После для выявления аномалий воспользовался функциями max и min, которые показали максимальные и минимальные значения всех столбцов датасета. Аномалии не были выявлены.

Решил изменить возраст, переведя его в категориальный тип, после чего добавил новый столбец, а старый удалил.

Заного обучил модель, метрика roc_auc_score показала снова 1, что говорит об идельной модели.

В результате работы метрика на обоих моделях показала идеальный результат, при этом 2 модель была более полной и четкой. 

Я считаю, что данный способ может быть использован при присутствии небольшого количесвта пропусков, либо если пропуски не являются ошибкой. В таком случае можно заполнить их некоторыми значениями и скорее всего полученная модель будет работать более качесвтвенно,чем если бы были удалены столбцы или объекты с пропущенными значниями.